In [ ]:
# 문제: 심장병 데이터셋에서 Z-score가 3보다 큰 이상치가 가장 많이 나타나는 컬럼과 그 컬럼의 이상치 개수 구하기

import pandas as pd
import numpy as np

df = pd.read_csv("/content/drive/MyDrive/datasets/heart_disease_data.csv")

# 결과 컬럼과 이상치 개수가 저장될 변수
result_col = ''
result_out = 0

# 컬럼별로 Z-score가 3보다 큰 이상치를 구해서 가장 개수가 많은 컬럼 저장
for i in df.columns:
  col_mean = df[i].mean()
  col_std = df[i].std()

  # Z-score 연산
  z_score = np.abs( (df[i] - col_mean) / col_std )

  # Z-score가 3보다 큰 이상치의 개수 저장
  outlier = df[z_score > 3]

  # 이상치가 가장 많은 컬럼과 이상치 개수 저장
  if len(outlier) > result_out:
    result_col = i
    result_out = len(outlier)

# 결과 출력
print(result_col)
print(result_out)

ca
18


In [ ]:
# 문제: 심장병 데이터셋에서 'chol' 컬럼의 Z-score를 구한 후 3보다 큰 이상치의 개수와 그 이상치 중 가장 큰 이상치의 값 구하기
# (가장 큰 이상치의 값은 절대값을 기준으로 구하며, 반올림하여 소수점 3자리까지 출력한다.)

import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/datasets/heart_disease_data.csv")

from scipy.stats import zscore

# Z-score로 만든 후 새로운 컬럼에 넣기
df['chol_z'] = zscore(df['chol'])

# Z-score가 3보다 큰 이상치 개수 저장
outlier_cnt = df['chol_z'][abs(df['chol_z']) > 3].count()

# Z-score가 3보다 큰 이상치 중 가장 큰 값 저장
outlier_value = df['chol_z'][abs(df['chol_z']) > 3].sort_values(ascending=False).iloc[0]

# 결과 출력
print(outlier_cnt)
print(round(outlier_value, 3))

13
6.167


In [ ]:
# 문제: 심장병 데이터셋에서 IQR 연산으로 ca 컬럼의 이상치 개수 구하기

import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/datasets/heart_disease_data.csv")

# 1사분위수, 3사분위수 저장
q1 = df['ca'].quantile(0.25)
q3 = df['ca'].quantile(0.75)

#  IQR, 1사분위수 하한값, 3사분위수 상한값 저장
iqr = q3 - q1
low = q1 - 1.5 * iqr
upp = q3 + 1.5 * iqr

# 이상치만 저장 후 개수 출력
outlier = df['ca'][ (df['ca'] < low) | (df['ca'] > upp) ]
print(len(outlier))

87


In [ ]:
# 문제: 각 구급 보고서별 출동시각과 신고시각의 차이를 '소요시간' 칼럼으로 만들어 초(sec) 단위로 구하고
# 소방서명별 소요시간의 평균을 오름차순으로 정렬했을 때, 3번째로 작은 소요시간의 값과 소방서명을 출력하시오.

import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/datasets/report_data.csv")

# 날짜형으로 만든 '신고일시' 컬럼 생성
df['신고일시'] = pd.to_datetime(df['신고일자'].astype(str) + ' ' + df['신고시각'].astype(str).str.zfill(6), format='%Y%m%d %H%M%S')

# 날짜형으로 만든 '출동일시' 컬럼 생성
df['출동일시'] = pd.to_datetime(df['출동일자'].astype(str) + ' ' + df['출동시각'].astype(str).str.zfill(6), format='%Y%m%d %H%M%S')

# 신고 및 출동 간격을 초 단위로 구한 '소요시간' 컬럼 생성
df['소요시간'] = (df['출동일시'] - df['신고일시']).dt.total_seconds()

# 소방서명별 소요시간 평균 계산 후 오름차순 정렬한 데이터프레임 생성
fire_station_time = df.groupby('소방서명')['소요시간'].mean().reset_index().sort_values(by='소요시간')

# 3번째로 작은 소요시간과 소방서명 출력
print(fire_station_time.iloc[2]['소방서명'])
print(fire_station_time.iloc[2]['소요시간'])

종로소방서
175.5


In [18]:
# 문제: airquality 데이터를 활용하여 결측값이 가장 많은 컬럼명을 확인하여 결측값을 최빈값으로 대체한 후
# 결측값을 대체하기 전의 컬럼 평균과 후의 컬럼 평균의 차이 구하기 (반올림하여 소수점 3자리까지 출력한다.)

import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/datasets/airquality.csv")

# 결측값이 가장 많은 컬럼명 저장
miss = df.isnull().sum().sort_values(ascending=False).index[0]

# 결측값 대체 전 컬럼 평균 저장
before = df[miss].mean()

# 결측값 최빈값으로 대체하기
df[miss].fillna(df[miss].mode().iloc[0], inplace=True)

# 결측값 대체 후 컬럼 평균 저장
after = df[miss].mean()

# 차이를 구한 후 반올림하여 소수점 3자리까지 출력
result = round(abs(before-after), 3)
print(result)

4.626
